## 80. Shuffle Hash Join & Broadcast Hash Join:

In [2]:
%load_ext autoreload
%autoreload 2

import os
path = os.environ.get("HOME")
os.chdir(f"{path}/github/pyspark-hand-on")
from src.utils.utils import spark_session
from pyspark.sql.functions import col, broadcast

spark = spark_session()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
workers = spark.read.parquet("./data/empleados.parquet")
workers.show()

+------+--------+
|nombre|num_dpto|
+------+--------+
|  Luis|      33|
| Katia|      33|
|  Raul|      34|
| Pedro|       0|
| Laura|      34|
|Sandro|      31|
+------+--------+



In [5]:
departments = spark.read.parquet("./data/departamentos.parquet")
# departments = departments.withColumn("num_dpto", col("id"))
departments.show()

+---+-----------+
| id|nombre_dpto|
+---+-----------+
| 31|     letras|
| 33|    derecho|
| 34| matemática|
| 35|informática|
+---+-----------+



In [8]:
df_dupl = workers.join(broadcast(departments), workers["num_dpto"] == departments["id"])
# df_dupl.printSchema()
df_dupl.show()

+------+--------+---+-----------+
|nombre|num_dpto| id|nombre_dpto|
+------+--------+---+-----------+
|  Luis|      33| 33|    derecho|
| Katia|      33| 33|    derecho|
|  Raul|      34| 34| matemática|
| Laura|      34| 34| matemática|
|Sandro|      31| 31|     letras|
+------+--------+---+-----------+



In [9]:
workers.join(broadcast(departments), workers["num_dpto"] == departments["id"]).explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- BroadcastHashJoin [num_dpto#1L], [id#36L], Inner, BuildRight, false
   :- Filter isnotnull(num_dpto#1L)
   :  +- FileScan parquet [nombre#0,num_dpto#1L] Batched: true, DataFilters: [isnotnull(num_dpto#1L)], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/home/ben/github/pyspark-hand-on/data/empleados.parquet], PartitionFilters: [], PushedFilters: [IsNotNull(num_dpto)], ReadSchema: struct<nombre:string,num_dpto:bigint>
   +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, bigint, false]),false), [plan_id=327]
      +- Filter isnotnull(id#36L)
         +- FileScan parquet [id#36L,nombre_dpto#37] Batched: true, DataFilters: [isnotnull(id#36L)], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/home/ben/github/pyspark-hand-on/data/departamentos.parquet], PartitionFilters: [], PushedFilters: [IsNotNull(id)], ReadSchema: struct<id:bigint,nombre_dpto:string>




In [22]:
df2 = workers.join(departments, "num_dpto")
df2.printSchema()

root
 |-- num_dpto: long (nullable = true)
 |-- nombre: string (nullable = true)
 |-- id: long (nullable = true)
 |-- nombre_dpto: string (nullable = true)



In [24]:
df3 = workers.join(departments, ["num_dpto"])
df3.show()

+--------+------+---+-----------+
|num_dpto|nombre| id|nombre_dpto|
+--------+------+---+-----------+
|      31|Sandro| 31|     letras|
|      33| Katia| 33|    derecho|
|      33|  Luis| 33|    derecho|
|      34| Laura| 34| matemática|
|      34|  Raul| 34| matemática|
+--------+------+---+-----------+

